In [1]:
import sys
sys.path.append("..")

from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer, DataCollatorWithPadding

from resilient_nlp.perturbers import WordScramblerPerturber

In [2]:
#imdb = load_from_disk("../data/imdb")
imdb = load_dataset('../output/huggingface/imdb')

Using custom data configuration redacted--imdb-f63738dec0d5e230
Reusing dataset parquet (/home/ec2-user/.cache/huggingface/datasets/parquet/redacted--imdb-f63738dec0d5e230/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
imdb

DatasetDict({
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    attack_eval_truncated: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 24000
    })
})

In [4]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [5]:
max_sequence_length = 128
batch_size = 32
eval_steps = 100
learning_rate=2e-05
num_train_epochs=5
output_dir = "../output/"
model_dir = "../models/"
early_stopping_patience = 10

In [6]:
wsp = WordScramblerPerturber(perturb_prob=0.2, weight_add=1.0, weight_drop=1.0, weight_swap=1.0,
                             weight_split_word=1.0, weight_merge_words=1.0, start_char_present=True,
                             end_char_present=True)

In [7]:
def tokenize_function(example):
    return tokenizer(example["text"],  truncation=True, padding="max_length", max_length=max_sequence_length )

def perturb_function(example):
    example["text"] = wsp.perturb(example["text"])

perturbed_datasets = imdb.map(perturb_function)
tokenized_datasets = perturbed_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/parquet/redacted--imdb-f63738dec0d5e230/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-bae2e0117af04f74.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/parquet/redacted--imdb-f63738dec0d5e230/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-723d494e7290a5a8.arrow
Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/parquet/redacted--imdb-f63738dec0d5e230/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-540b8406383736c0.arrow


In [8]:
tokenized_datasets = tokenized_datasets.remove_columns(["text", ])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [9]:
# from torch.utils.data import DataLoader

# train_dataloader = DataLoader(
#     tokenized_datasets["train"], shuffle=True, batch_size=16, collate_fn=data_collator
# )
# dev_dataloader = DataLoader(
#     tokenized_datasets["dev"], batch_size=16, collate_fn=data_collator
# )

In [10]:
# for batch in train_dataloader:
#     break
# {k: v.shape for k, v in batch.items()}

In [11]:
from transformers import AutoModelForSequenceClassification, BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
from transformers import TrainingArguments
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

In [13]:
# def compute_metrics(eval_preds):
#     metric = load_metric("imdb)
#     logits, labels = eval_preds
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [14]:

# training_args = TrainingArguments('bert-base-uncased-imdb', 
#                                   learning_rate=2e-05,
#                                   per_device_train_batch_size =batch_size, 
#                                   per_device_eval_batch_size =batch_size,

#                           )

training_args = TrainingArguments(
   output_dir+"bert-base-uncased-imdb",
   evaluation_strategy ='steps',
   eval_steps = eval_steps , # Evaluation and Save happens every eval_steps steps
   save_total_limit = 1, # Only last  model is saved. Older ones are deleted.
   learning_rate=learning_rate,
   per_device_train_batch_size=batch_size,
   per_device_eval_batch_size=batch_size,
   num_train_epochs=num_train_epochs,
   metric_for_best_model = 'f1',
   load_best_model_at_end=True)

In [15]:
from transformers import Trainer, EarlyStoppingCallback

In [16]:

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)],
    compute_metrics=compute_metrics,
)


In [17]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Runtime,Samples Per Second
100,No log,0.392239,0.835000,0.821918,0.850202,0.835821,2.096100,477.072000
200,No log,0.343418,0.845000,0.877506,0.797571,0.835631,2.116700,472.430000
300,No log,0.347368,0.860000,0.862705,0.852227,0.857434,2.113700,473.102000
400,No log,0.310695,0.860000,0.819495,0.919028,0.866412,2.121700,471.321000
500,0.361200,0.306364,0.865000,0.845857,0.888664,0.866732,2.124000,470.804000
600,0.361200,0.313365,0.869000,0.853801,0.886640,0.869911,2.126400,470.270000
700,0.361200,0.328975,0.864000,0.903153,0.811741,0.855011,2.130500,469.369000
800,0.361200,0.334307,0.868000,0.891775,0.834008,0.861925,2.128200,469.876000
900,0.361200,0.342044,0.876000,0.851711,0.906883,0.878431,2.138700,467.569000
1000,0.259000,0.326736,0.870000,0.866935,0.870445,0.868687,2.134100,468.586000


TrainOutput(global_step=2900, training_loss=0.19107767960120892, metrics={'train_runtime': 863.924, 'train_samples_per_second': 4.526, 'total_flos': 7796898636582912, 'epoch': 3.71})

NameError: name 'model' is not defined

In [19]:
predictions = trainer.predict(tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(24000, 2) (24000,)


In [20]:
from sklearn.metrics import classification_report
preds = np.argmax(predictions.predictions, axis=-1)

In [21]:
print(classification_report(predictions.label_ids, preds))

              precision    recall  f1-score   support

           0       0.91      0.85      0.88     11994
           1       0.86      0.92      0.89     12006

    accuracy                           0.89     24000
   macro avg       0.89      0.89      0.89     24000
weighted avg       0.89      0.89      0.89     24000

